# Putting guardrails around LLM input and LLM output

Guardrails AI is a package which is integrated with LangChain : "pip install guardrails-ai"

You need an account and an API key. You then need to install the validators that you want to use from the CLI

ToxicLanguage: Detects and optionally removes toxic or inappropriate language from the output.

guardrails hub install hub://guardrails/toxic_language --quiet

In [8]:
from guardrails import Guard
from guardrails.hub import ToxicLanguage

In [9]:
guard = Guard().use(ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception")

In [10]:
try:
    # Test failing response
    guard.validate(
        "Please look carefully. You are a stupid idiot who can't do anything right."
    )
except Exception as e:
    print(e)

Validation failed for field with errors: The following sentences in your response were found to be toxic:

- You are a stupid idiot who can't do anything right.


Check for personal information

guardrails hub install hub://guardrails/detect_pii

In [14]:
from guardrails import Guard
from guardrails.hub import DetectPII

In [15]:
guard = Guard().use(DetectPII, ["EMAIL_ADDRESS", "PHONE_NUMBER"], "exception")

In [16]:
try:
    guard.validate(
        "If interested, apply at not_a_real_email@guardrailsai.com"
    )  # Validator fails
except Exception as e:
    print(e)

Validation failed for field with errors: The following text in your response contains PII:
If interested, apply at not_a_real_email@guardrailsai.com


Now using to filter out specific words eg competitor names

Example where integrated with a LLM on the OUTPUT

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model='gpt-4o-mini')

In [18]:
from guardrails import Guard
from guardrails.hub import CompetitorCheck
competitors_list = ["Delta Air Lines", "American Airlines", "United Airlines"]
guard = Guard().use(CompetitorCheck, competitors=competitors_list, on_fail="fix")

In [19]:
prompt = ChatPromptTemplate.from_template("Answer this question {question}")
output_parser = StrOutputParser()
chain = prompt | model | guard.to_runnable() | output_parser

In [20]:
chain.invoke({"question": "What are the top five airlines for domestic travel in the US?"})

C:\Users\patri\anaconda3\envs\lg311\Lib\site-packages\guardrails\validator_service\__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


"As of my last update, the top five airlines for domestic travel in the U.S. are typically considered to be:\n\n1.**[COMPETITOR]**\n2.**[COMPETITOR]**\n3.**Southwest Airlines**\n4.**[COMPETITOR]**\n5.**Alaska Airlines**\n\nThese rankings can vary based on different metrics such as customer satisfaction, flight availability, on-time performance, and overall market share.It's always a good idea to check current rankings and customer reviews for the latest information."

CompetitorCheck: Identifies and optionally removes mentions of specified competitor names

guardrails hub install hub://guardrails/competitor_check --quiet

Example where integrated with a LLM on the INPUT

In [20]:
prompt = ChatPromptTemplate.from_template("Answer this question {question}")
output_parser = StrOutputParser()
chain = prompt | guard.to_runnable() | model | output_parser

In [23]:
try:
    chain.invoke({"question": "Who owns not_a_real_email@guardrailsai.com?"})
except Exception as e:
    print(e)

C:\Users\patri\anaconda3\envs\lg311\Lib\site-packages\guardrails\validator_service\__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


Validation failed for field with errors: The following text in your response contains PII:
messages=[HumanMessage(content='Answer this question Who owns not_a_real_email@guardrailsai.com?', additional_kwargs={}, response_metadata={})]
